# Regression using Cyclic Boosting

First, install the  package and its dependencies

```sh
!pip install cyclic-boosting
```

In [ ]:
import pandas as pd
import numpy as np
import os
import datetime

# Let's use the test dataset from kaggle

Sign in to Kaggle at the URL below and download the dataset.  
https://www.kaggle.com/datasets/lakshmi25npathi/bike-sharing-dataset

Place the downloaded dataset in the following directory.  
examples/regression/tornado/test_1/bike_sharing_data/

For time-series data, a "date" column must be included to indicate the date and time the data was obtained. The column name and format must be consistent. The "dayofweek" column for the day of the week and the "dayofyear" column for the total number of days in the year are automatically created if not already present, but if they are already present, the column names must be correct.

This dataset has hourly data. In this dataset, the "instant" column is the data number. The "casual" and "registered" columns are the breakdown of sales, so they should be deleted.

In [ ]:
parpath = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
df = pd.read_csv(parpath + "/bike_sharing_data/hour.csv")
df = df.rename(columns={'dteday': 'date', 'weekday': 'dayofweek'})
df = df.drop(columns=['instant', 'casual', 'registered'])
df["date"] = pd.to_datetime(df["date"])
df['date'] = df['date'] + df['hr'].map(lambda x: datetime.timedelta(hours=float(x)))

df.to_csv("./bike_sharing_hour.csv", index=False)

In [ ]:
df.head()

# Automated Machine Learning with Tornado
With tornado, you can automatically perform data preparation, feature property setting, hyperparameter tuning, model building, training, evaluation, and plotting!

In [4]:
from cyclic_boosting.tornado import Generator, Manager, Trainer

data_deliverler = Generator.TornadoDataModule("./bike_sharing_hour.csv")
manager = Manager.TornadoVariableSelectionModule()
trainer = Trainer.SqueezeTrainer(data_deliverler, manager)
trainer.run(target="cnt", log_policy="compute_COD", verbose=False)

# Load the best model and make predictions.

Get the best model path.

In [ ]:
import pickle
from pathlib import Path

model_nos = []
for p in sorted(Path("./models/").glob("model*")):
    model_nos.append(str(p)[str(p).find("_") + 1 :])
model_path = f"./models/model_{model_nos[-1]}/model_{model_nos[-1]}.pkl"
print(model_path)

Make predictions with the best model.

In [ ]:
data = {
    'season': [4],
    'yr': [0],
    'mnth': [11],
    'hr': [18],
    'holiday': [0],
    'workingday': [1],
    'weathersit': [2],
    'temp': [0.341667],
    'atemp': [0.323221],
    'hum': [0.575833],
    'windspeed': [0.305362],
    'dayofweek': [4],
    'dayofyear': [180],
}
X = pd.DataFrame(data)

with open(model_path, "rb") as f:
    CB_est = pickle.load(f)
    yhat = CB_est.predict(X.copy())
    print(yhat)